# Automated ML

## Dependencies 

All the dependencies needed to complete the project appear here.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.steps import AutoMLStep

from azureml.widgets import RunDetails

import joblib

from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


## Workspace

The `config.json` file is downloaded from Azure environment and has to be in the project folder in order for this cell to run.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

HeartFailurePrediction
MajorResource
eastus2
e4ad9e79-a431-462c-9f80-d219f4225e07


## Create an Azure ML experiment
I am creating an experiment named `heart-failure-prediction` and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.


In [3]:
# Choose a name for the run history container in the workspace.

experiment_name = 'heart-failure-prediction'
project_folder = './major-project'

experiment = Experiment(ws, experiment_name)
experiment

run = experiment.start_logging()

## Create or Attach a cluster

We will need to create a [compute target](https://docs.microsoft.com/en-us/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) for the AutoML run. In case the compute target (named `compute-cluster` in this script) is not found, a new one is created using the default AmlCompute as the training compute resource.

In [4]:
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "compute-cluster1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    # Poll for a minimum number of nodes (min_nodes = 1). 
    # If no min node count is provided it uses the scale settings for the cluster.
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D11_V2', min_nodes=0, max_nodes=3)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-04-14T23:43:11.608000+00:00', 'errors': None, 'creationTime': '2023-04-14T21:46:12.721227+00:00', 'modifiedTime': '2023-04-14T21:49:25.360021+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 3, 'nodeIdleTimeBeforeScaleDown': 'PT2000S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D11_V2'}


## Dataset

### Overview

The dataset used is taken from [Kaggle](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) and the data comes from 299 patients with heart failure collected at the Faisalabad Institute of Cardiology and at the Allied Hospital in Faisalabad (Punjab, Pakistan), during April–December 2015. The patients consisted of both women (105) and men (194), and the main task of the project is to classify the patients based on their odds of survival.

Dataset features:

| Feature | Explanation |
| :---: | :---: |
| *age* | Age of patient |
| *anaemia* | Decrease of red blood cells or hemoglobin |
| *creatinine-phosphokinase* | Level of the CPK enzyme in the blood |
| *diabetes* | Whether the patient has diabetes or not |
| *ejection_fraction* | Percentage of blood leaving the heart at each contraction |
| *high_blood_pressure* | Whether the patient has hypertension or not |
| *platelets* | Platelets in the blood |
| *serum_creatinine* | Level of creatinine in the blood |
| *serum_sodium* | Level of sodium in the blood |
| *sex* | Female (F) or Male (M) |
| *smoking* | Whether the patient smokes or not |
| *time* | Follow-up period |
| *DEATH_EVENT* | Whether the patient died during the follow-up period |


In [5]:
data = pd.read_csv('./heart_failure_clinical_records_dataset.csv')

found = False
key = "heart-failure-risk"
description_text = "Dataset for Heart Failure Risk Prediction"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        my_dataset = 'https://github.com/HumdaanSyed/Heart-Failure-Risk-Prediction/blob/0fb86b13fda62c62fd4cb2542f65d8fb27327304/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(my_dataset)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
                                
# Preview of the first five rows
print(data.head())

# Explore data
print(data.describe())

df = dataset.to_pandas_dataframe()
df.describe()

# Data columns
df.columns = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time', 'DEATH_EVENT']
x = df[['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']]
y = df[['DEATH_EVENT']]


    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.7           116    0   

   smoking  time  DEATH_EVENT  
0        0     4            1  
1        0     6            1  
2       

## AutoML Configuration

Here is an overview of the `automl` settings and configuration I used for the AutoML run:

`"n_cross_validations": 2`

This parameter sets how many cross validations to perform, based on the same number of folds (number of subsets). As one cross-validation could result in overfit, in my code I chose 2 folds for cross-validation; thus the metrics are calculated with the average of the 2 validation metrics.

`"primary_metric": 'accuracy'`

I chose accuracy as the primary metric as it is the default metric used for classification tasks.

`"enable_early_stopping": True`

It defines to enable early termination if the score is not improving in the short term. In this experiment, it could also be omitted because the _experiment_timeout_minutes_ is already defined below.

`"max_concurrent_iterations": 4`

It represents the maximum number of iterations that would be executed in parallel.

`"experiment_timeout_minutes": 20`

This is an exit criterion and is used to define how long, in minutes, the experiment should continue to run. To help avoid experiment time out failures, I used the value of 20 minutes.

`"verbosity": logging.INFO`

The verbosity level for writing to the log file.

`compute_target = compute_target`

The Azure Machine Learning compute target to run the Automated Machine Learning experiment on.

`task = 'classification'`

This defines the experiment type which in this case is classification. Other options are _regression_ and _forecasting_.

`training_data = dataset`

The training data to be used within the experiment. It should contain both training features and a label column - see next parameter.

`label_column_name = 'DEATH_EVENT'` 

The name of the label column i.e. the target column based on which the prediction is done.

`path = project_folder`

The full path to the Azure Machine Learning project folder.

`featurization = 'auto'`

This parameter defines whether featurization step should be done automatically as in this case (_auto_) or not (_off_).

`debug_log = 'automl_errors.log`

The log file to write debug information to.

`enable_onnx_compatible_models = False`

I chose not to enable enforcing the ONNX-compatible models at this stage. However, I will try it in the future. For more info on Open Neural Network Exchange (ONNX), please see [here](https://docs.microsoft.com/en-us/azure/machine-learning/concept-onnx).


In [11]:
# Automl settings

automl_settings = {"n_cross_validations": 2,
                    "primary_metric": 'accuracy',
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 35,
                    "verbosity": logging.INFO
                    }

# Parameters for AutoMLConfig

automl_config = AutoMLConfig(compute_target = compute_target,
                            task='classification',
                            training_data=dataset,
                            label_column_name='DEATH_EVENT',
                            path = project_folder,
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            enable_onnx_compatible_models=False,
                            **automl_settings
                            )

In [12]:
# Submit the experiment

remote_run = experiment.submit(automl_config, show_output = True)
remote_run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on compute-cluster1 with default configuration
Running on remote compute: compute-cluster1


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-prediction,AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTI

{'runId': 'AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b',
 'target': 'compute-cluster1',
 'status': 'Completed',
 'startTimeUtc': '2023-04-15T22:20:09.778216Z',
 'endTimeUtc': '2023-04-15T22:27:18.687658Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster1',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-prediction","subscription_id":"e4ad9e79-a431-462c-9f80-d219f4225e07","resource_group":"MajorResource","workspace_name":"HeartFailurePrediction","region":"eastus2","compute_target":"compute-clu

In [13]:
# get_status()
# Fetch the latest status of the run. It should show 'Completed'

print("Run Status: ",remote_run.get_status())

Run Status:  Completed


## Run Details

In the cell below, I use the `RunDetails` widget and show the children runs of the experiment.

In [14]:

RunDetails(remote_run).show()

# Get details from each run
for child_run in remote_run.get_children():
    print('===================================================')
    print(child_run)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Run(Experiment: heart-failure-prediction,
Id: AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_38,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: heart-failure-prediction,
Id: AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_37,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: heart-failure-prediction,
Id: AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_36,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: heart-failure-prediction,
Id: AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_35,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: heart-failure-prediction,
Id: AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_34,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: heart-failure-prediction,
Id: AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_33,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: heart-failure-prediction,
Id: AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_32,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: heart-failure-predict

## Best Model
In the cell below, I get the best model from the automl experiment and display all the properties of the model.

In [15]:

best_run, fitted_model = remote_run.get_output()

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
print('===================================================')

# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
print('===================================================')

# get_properties()
# Fetch the latest properties of the run from the service
print("Best run properties :",best_run.get_properties())
print('===================================================')



Best run metrics : {'norm_macro_recall': 0.6350340009608225, 'log_loss': 0.3876399499599096, 'AUC_micro': 0.9108674223883808, 'precision_score_micro': 0.8629082774049217, 'balanced_accuracy': 0.8175170004804113, 'f1_score_micro': 0.8629082774049217, 'average_precision_score_micro': 0.9093184202163714, 'AUC_macro': 0.9006630037732388, 'recall_score_micro': 0.8629082774049217, 'matthews_correlation': 0.6799427752774265, 'average_precision_score_macro': 0.8858107489697452, 'recall_score_macro': 0.8175170004804113, 'weighted_accuracy': 0.8981972119791409, 'accuracy': 0.8629082774049217, 'f1_score_macro': 0.8325528093689437, 'precision_score_macro': 0.8650294150294151, 'recall_score_weighted': 0.8629082774049217, 'average_precision_score_weighted': 0.9073759312902485, 'AUC_weighted': 0.9006630037732388, 'f1_score_weighted': 0.8578559922118096, 'precision_score_weighted': 0.8659450268846243, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_

In [16]:
best_run.get_file_names()

# Download the yaml file that includes the environment dependencies
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')


In [17]:
# Download the model file

best_run.download_file('outputs/model.pkl', 'Automl_model.pkl')

In [18]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('21', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.3, gamma=0, max_depth=10, max_leaves=511, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=2.1875, reg_lambda=0.4166666666666667, subsample=0.5, tree_method='auto'))], verbose=False)), 

In [19]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-prediction,AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_37,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [20]:
# Save the best model

best_run.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')

print(best_run)


Run(Experiment: heart-failure-prediction,
Id: AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_37,
Type: azureml.scriptrun,
Status: Completed)


## Best Model Based on Another Metric

Show the run and model that has the highest **AUC_weighted** and the one with the smallest **average_precision_score_weighted** value:

In [21]:
lookup_metric = "AUC_weighted"
best_run, fitted_model = remote_run.get_output(metric = lookup_metric)
print('========================================================')
print("Based on AUC_weighted: ",best_run)
print(fitted_model)

lookup_metric = "average_precision_score_weighted"
best_run, fitted_model = remote_run.get_output(metric = lookup_metric)
print('========================================================')
print("Based on average_precision_score_weighted: ",best_run)
print(fitted_model)

Based on AUC_weighted:  Run(Experiment: heart-failure-prediction,
Id: AutoML_19c0a6e2-24a1-4698-b53b-122c354d1c1b_11,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                                        class_weight='balanced',
                                        criterion='gini', max_depth=None,
                                        max_features='sqrt',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samp

## Model Deployment

As the best model coming from AutoML run has better accuracy than the one coming from the HyperDrive run, I deploy it in the cell below, register it, create an inference config and deploy the model as a web service.

In [22]:
model = remote_run.register_model(model_name = 'best_run_automl.pkl')
print(remote_run.model_id)

environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

# Deploying the model via ACI WebService
# https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/machine-learning/how-to-deploy-azure-container-instance.md

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)


best_run_automl.pkl
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-04-15 23:12:09+00:00 Creating Container Registry if not exists..
2023-04-15 23:22:09+00:00 Registering the environment.
2023-04-15 23:22:10+00:00 Use the existing image.
2023-04-15 23:22:11+00:00 Submitting deployment to compute..
2023-04-15 23:22:16+00:00 Checking the status of deployment aciservice..
2023-04-15 23:23:50+00:00 Checking the status of inference endpoint aciservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [26]:
# Getting the service state
# The scorig URI & the primary authentication key are copied to the endpoint.py file in order to test the deployed service.
# The Swagger URI can be used in Swagger UI: https://petstore.swagger.io/ For more info, please see the relevant part in the README file.

# Authentication is enabled, so I use the get_keys method to retrieve the primary and secondary authentication keys:
primary, secondary = service.get_keys()

print('Service state: ' + service.state)
print('Service scoring URI: ' + service.scoring_uri)
print('Service Swagger URI: ' + service.swagger_uri)
print('Service primary authentication key: ' + primary)


Service state: Healthy
Service scoring URI: http://c0f5aceb-a6f6-46e9-97d1-3e58d5bbd7e2.eastus2.azurecontainer.io/score
Service Swagger URI: http://c0f5aceb-a6f6-46e9-97d1-3e58d5bbd7e2.eastus2.azurecontainer.io/swagger.json
Service primary authentication key: 6t3Ag7kMfSONfkIOsdp3NfRxSkf1WSL6


In [28]:
# Sending a request to the deployed web service to test it: consuming model endpoint

%run endpoint.py

{"result": [1, 1]}
++++++++++++++++++++++++++++++
Expected result: [true, true], where 'true' means '1' as result in the 'DEATH_EVENT' column


In [29]:
# Printing the logs
print(service.get_logs())

2023-04-15T23:23:38,645317900+00:00 - rsyslog/run 
2023-04-15T23:23:38,660956800+00:00 - gunicorn/run 
2023-04-15T23:23:38,673375900+00:00 | gunicorn/run | 
2023-04-15T23:23:38,677508800+00:00 - nginx/run 
2023-04-15T23:23:38,677825600+00:00 | gunicorn/run | ###############################################
2023-04-15T23:23:38,688820600+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-04-15T23:23:38,699993200+00:00 | gunicorn/run | ###############################################
2023-04-15T23:23:38,709553900+00:00 | gunicorn/run | 
2023-04-15T23:23:38,719716400+00:00 | gunicorn/run | 
2023-04-15T23:23:38,728059700+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230324.v2
2023-04-15T23:23:38,732911800+00:00 | gunicorn/run | 
2023-04-15T23:23:38,735185700+00:00 | gunicorn/run | 
2023-04-15T23:23:38,740173000+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

## Deleting the service
Putting the deletion of the service in a separate cell to avoid accidentally running the cell before finishing the tasks

In [ ]:

service.delete()
